In [2]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!unzip model.zip

Archive:  model.zip
replace model/final/adapter_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import pandas as pd
import string
import re
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Import the pretrained model

In [5]:
output_dir = 'model'
save_dir = f'./{output_dir}/final/'

In [6]:
AutoModelForSequenceClassification.from_pretrained('roberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [7]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained(save_dir, local_files_only=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**clean text**

In [8]:
def remove_html(text) :
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

In [9]:
def remove_mentions(text):
    mention = "@[A-Za-z0-9_]+"
    return re.sub(mention,"", text)

In [10]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [11]:
def clean_text(text) :
    text = str(text).lower()
    text = remove_mentions(text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = remove_html(text)
    text = remove_emojis(text)
    return text

In [12]:
df_test = pd.read_csv('test.csv')

In [13]:
#All columns uppercase
df_test.columns = [col.upper() for col in df_test.columns]

In [14]:
df_test.drop(['ID', 'KEYWORD', 'LOCATION'], axis=1, inplace=True)

In [15]:
df_test['FINAL_TEXT'] = df_test['TEXT'].apply( lambda x:clean_text(x))

In [16]:
df_test.drop(['TEXT'], axis=1, inplace=True)

In [17]:
df_test.rename(
    {'FINAL_TEXT': 'text'}, axis=1, inplace=True)

In [18]:
model_id = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [19]:
input_texts = df_test['text'].to_list()

In [21]:
len(input_texts[:500])

500

In [ ]:
import torch

input_texts = df_test['text'].to_list()

# Tokenize the input texts
inputs = tokenizer(input_texts, return_tensors="pt", padding="max_length", truncation=True)



In [23]:
inputs

{'input_ids': tensor([[    0,  8987,  1102,  ...,     1,     1,     1],
        [    0, 42225,    59,  ...,     1,     1,     1],
        [    0,  8585,    16,  ...,     1,     1,     1],
        ...,
        [    0, 12354,   516,  ...,     1,     1,     1],
        [    0, 16258,   743,  ...,     1,     1,     1],
        [    0, 14853,  1116,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
# Make predictions
with torch.no_grad():
    outputs = finetuned_model(**inputs)
    logits = outputs.logits


In [ ]:

# Convert logits to probabilities using softmax
probs = torch.nn.functional.softmax(logits, dim=1)

y_pred = []
# Print the predicted classes and probabilities for each text
for i, text in enumerate(input_texts):
    predicted_class = torch.argmax(probs[i]).item()
    y_pred.append(predicted_class)